In [20]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import Optional, Literal
import joblib
import uvicorn
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer


In [21]:
lr_from_joblib = joblib.load('My_model.pkl')
encod = joblib.load("Encoded.pkl")

In [22]:
class Property(BaseModel):
    region: str
    property_type: str
    habitable_surface: float
    number_of_rooms: int
    has_terrace: str
    has_garden: str
    is_furnished: str
    has_swimming_pool: str
    has_open_fire: str
    

In [23]:
app = FastAPI()

In [24]:
# Route
@app.get('/')
def get_request():
    return "status alive"

In [25]:
@app.post("/prediction")
def price_prediction(data_input: Property):
    try:
        # get the input data
        data = pd.DataFrame([data_input.dict()])
        data = data.replace({"Yes": 1, "No": 0})
        data_encoded = encod.transform(data)

       
        # my prediction - with my model
        prediction = lr_from_joblib.predict(data_encoded)
        # dict for the response
        response = {'prediction': float(prediction[0])}

        return response
  
    except KeyError as e:
        raise HTTPException(status_code = 500, detail= str(e))
    

In [ ]:
if __name__ == "__main__":
    uvicorn.run(app, host="127.0.0.1", port=5000)